In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  ChatPromptTemplate
from langchain.callbacks import  StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system" , "You are a world-class international chef. you create esay to follow recipies for any type of cuisine with easy to find ingredients. ",),
    ("human" , "I want to cook {cuisine} food")
])

chef_chain = chef_prompt | chat


In [19]:



veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain  

final_chain.invoke({
    "cuisine" : "indian"
})

veg_chain.invoke({
    "recipe" : "chatmodel"
})


Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Here is a simple recipe for Chicken Tikka Masala, a popular Indian dish that you can easily make at home.

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon garam masala
- 1 teaspoon chili powder (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together yogurt, lemon juice, cumin, paprika, turmeric, coriander, garam masala, chili powder, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 ho

AIMessageChunk(content='I\'m sorry, but I\'m not familiar with a dish called "chatmodel." Could you please provide more information or clarify the name so I can assist you in finding a vegetarian alternative for it?')